In [2]:
! pip install requests

  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)


You should consider upgrading via the 'F:\GitHub\HSE\dev_ops\hw1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


6. **Разработка микросервиса предобработки данных**
   - Спроектируйте микросервис для предобработки данных, необходимых для работы вашей модели.
   - Реализуйте сервис с использованием **Flask**, **FastAPI** или другого подходящего фреймворка.
   - Оформите микросервис в виде **Docker-контейнера**.

In [10]:
import requests

url = "http://localhost:8500/preprocess"
files = {"file": open("test_data/test_image_1.png", "rb")}

response = requests.post(url, files=files)
response.json()

{'processed_image': [[[[1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0],
    [1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0],
    [1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     0.9843137264251709,
     0.9647058844566345,
     0.9647058844566345,
     0.9647058844566345,
     0.9647058844566345,
     0.9647058844566345,
     0.9647058844566345,
     0.9647058844566345,
     0.9882352948188782,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0,
     1.0],
   

In [20]:
import requests

url_preprocess = "http://localhost:8500/preprocess"
files = {"file": open("test_data/test_image_4.png", "rb")}

try: 
    response = requests.post(url_preprocess, files=files)
    processed_data = response.json()["processed_image"]

    url_triton = "http://localhost:8000/v2/models/pytorch_model/infer"

    payload = {
        "inputs": [{"name": "input", "shape": [1, 1, 28, 28], "datatype": "FP32", "data": processed_data}]
    }

    response = requests.post(url_triton, json=payload)
    print("Предсказание модели:", response.json())
except requests.exceptions.RequestException as e:
    print(f"Ошибка: {e}")

Предсказание модели: {'model_name': 'pytorch_model', 'model_version': '1', 'outputs': [{'name': 'output', 'datatype': 'FP32', 'shape': [1, 10], 'data': [-3.212080955505371, -5.790658950805664, -4.420777797698975, -0.7538982629776001, -6.495770454406738, -2.3953492641448975, -5.863797664642334, -4.248612880706787, -4.729637622833252, -1.9896405935287476]}]}
